In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [2]:
# Load the data
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

# Assign X and Y from training data
X_train = train.drop(labels = ["label"],axis = 1) 
y_train = train["label"].values

# Normalize the data for better performance
X_train = X_train / 255.0
test = test / 255.0

# Reshape image in 3 dimensions (height = 28px, width = 28px , channel = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)

# create validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.30)

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, kernel_size=(5, 5), activation="relu", strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu", strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation="softmax"),
    ]
)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 6, 6, 32)          0

In [4]:
# CREATE MORE IMAGES VIA DATA AUGMENTATION
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1)
datagen.fit(X_train)
batch_size = 64
batches = datagen.flow(X_train, y_train, batch_size=batch_size)
val_batches = datagen.flow(X_val, y_val, batch_size=batch_size)

In [5]:
logdir = "logs"
!rm -rf ./{logdir}
tensorboard_callback = TensorBoard(log_dir=logdir)
# Set a learning rate annealer
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
    patience=3,
    verbose=1,
    factor=0.5,
    min_lr=0.00001)

# Train the model
epochs = 50
history = model.fit(
    batches,
    epochs=epochs,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=val_batches,
    callbacks=[reduce_lr, tensorboard_callback])

Epoch 1/50
  1/459 [..............................] - ETA: 0s - loss: 2.4202 - accuracy: 0.0469WARNING:tensorflow:From /Users/hughpearse/Downloads/kaggle/digit-recognizer/sandbox/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
459/459 [==============================] - 66s 145ms/step - loss: 0.2993 - accuracy: 0.9055 - val_loss: 0.4734 - val_accuracy: 0.8515
Epoch 2/50
459/459 [==============================] - 61s 134ms/step - loss: 0.1062 - accuracy: 0.9684 - val_loss: 0.1120 - val_accuracy: 0.9661
Epoch 3/50
459/459 [==============================] - 65s 141ms/step - loss: 0.0809 - accuracy: 0.9771 - val_loss: 0.1019 - val_accuracy: 0.9694
Epoch 4/50
459/459 [==============================] - 143s 312ms/step - loss: 0.0717 - accuracy: 0.9793 - val_loss: 0.0826 - val_accuracy: 0.9749
Epoch 5/

459/459 [==============================] - 60s 130ms/step - loss: 0.0061 - accuracy: 0.9983 - val_loss: 0.0291 - val_accuracy: 0.9930
Epoch 47/50
459/459 [==============================] - 60s 132ms/step - loss: 0.0053 - accuracy: 0.9985 - val_loss: 0.0270 - val_accuracy: 0.9938
Epoch 48/50
459/459 [==============================] - 60s 130ms/step - loss: 0.0056 - accuracy: 0.9984 - val_loss: 0.0311 - val_accuracy: 0.9935
Epoch 49/50
459/459 [==============================] - 60s 132ms/step - loss: 0.0059 - accuracy: 0.9982 - val_loss: 0.0273 - val_accuracy: 0.9934
Epoch 50/50
459/459 [==============================] - 62s 134ms/step - loss: 0.0066 - accuracy: 0.9980 - val_loss: 0.0303 - val_accuracy: 0.9931


In [6]:
%tensorboard --logdir {logdir}

Reusing TensorBoard on port 6006 (pid 8809), started 1 day, 0:21:40 ago. (Use '!kill 8809' to kill it.)

In [7]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)